In [1]:
import pandas as pd
import numpy as np
from huggingface_hub import login, logout
from datasets import load_dataset
from util.util_data import tokenize
import requests, os, sys, sqlite3, time
import yfinance as yf

In [2]:
eng = sqlite3.connect('./src/data.db')
tix = pd.read_sql('SELECT DISTINCT ticker FROM catalysts', eng)
df = pd.read_sql('SELECT * FROM catalysts', eng)
tickers = yf.Tickers(' '.join(tix.ticker.tolist()))

In [3]:
names = list()
for t in tix.ticker.tolist():
    try:
        names.append([t, tickers.tickers[t].info['longName'].upper()])
    except Exception:
        names.append([t, None])

In [5]:
names = pd.DataFrame(names, columns=['ticker', 'full'])
names.to_csv('./out/full_names.csv', index=False)